# Real color farm polygon images

This notebook contains scripts for retrieving RGB images of farm polygons on specified dates to be used to compare satellite images with NDVI time-series curves.

In [1]:
import os
from pprint import pprint 

from dotenv import load_dotenv 
import ee
import geemap

/Users/rafidmahbub/Desktop/regen_analytics_app/.venv/lib/python3.11/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
load_dotenv()
GEE_PROJECT = os.getenv("GEE_PROJECT")

ee.Authenticate()
ee.Initialize(project=GEE_PROJECT)

In [3]:
def mask_s2_clouds(image: ee.Image) -> ee.Image:
    # This function performs cloud masking with Sentinel 2's Q60 band.
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10 # Opaque clouds
    cirrus_bit_mask = 1 << 11 # Cirrus clouds

    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0)
    )

    return image.updateMask(mask)

def get_image_dates(image_collection: ee.ImageCollection):

    # Generate timestamps from the image collection
    dates = (
        image_collection.aggregate_array("system:time_start")
                        .map(lambda time: ee.Date(time).format("YYYY-MM-dd"))
    )

    return dates.getInfo()

def get_rgb_image(geometry: ee.Geometry, START_DATE: str) -> ee.Image:
    """
    This function retrieves the least cloudy image of the given polygon
    at a specified 10-day window.
    """
    START_DATE = ee.Date(START_DATE)
    NEXT_DATE = START_DATE.advance(10, "day")

    # Get Sentinel-2 image collection, filter and sort by cloud cover
    s2 = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(geometry)
        .filterDate(START_DATE, NEXT_DATE)
        .map(mask_s2_clouds)
        .sort("CLOUD_COVER")
    )

    # Take the first (least cloudy) image
    image = ee.Image(s2.first())

    # Select RGB bands
    rgb = image.select(["B4", "B3", "B2"]).clip(geometry)

    return rgb.divide(10000)

In [4]:
from shapely import from_wkt

# Example polygon
wkt_polygon = "POLYGON ((34.989244937896736 0.9892645092090744, 34.98921275138856 0.9904343907706966, 34.98782873153687 0.9914647448315846, 34.987281560897834 0.9928385497472638, 34.986755847930915 0.9931820008870327, 34.98634815216065 0.9931605351918125, 34.986037015914924 0.9929780767770978, 34.98542547225953 0.9928922139901996, 34.98532891273499 0.9922375101668665, 34.985532760620124 0.9920765174035694, 34.98558640480042 0.9917867304098947, 34.98488903045655 0.9903055964905625, 34.9841594696045 0.989436234968717, 34.9856722354889 0.9882878187820331, 34.98659491539002 0.9882878187820331, 34.98835444450379 0.9891571806047718, 34.989244937896736 0.9892645092090744))"

def convert_wkt_to_ee_geometry(wkt: str) -> ee.Geometry:
    # Converts wkt polygons to ee.Geometry objects 
    shapely_polygon = from_wkt(wkt)
    x_coords, y_coords = shapely_polygon.exterior.coords.xy 

    xy_coords = [[x, y] for x, y in zip(x_coords, y_coords)]

    return ee.Geometry.Polygon(xy_coords)

In [9]:
polygon = convert_wkt_to_ee_geometry(wkt_polygon)
center = polygon.centroid().getInfo()["coordinates"]

# Sentinel-2 image collection filtered by region and date range
collection: ee.ImageCollection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(polygon)
    .filterDate("2023-01-01", "2023-12-31")
)

unique_dates = get_image_dates(collection)

pprint(unique_dates)

['2023-01-01',
 '2023-01-06',
 '2023-01-11',
 '2023-01-16',
 '2023-01-21',
 '2023-01-26',
 '2023-01-31',
 '2023-02-05',
 '2023-02-10',
 '2023-02-15',
 '2023-02-20',
 '2023-02-25',
 '2023-03-02',
 '2023-03-07',
 '2023-03-12',
 '2023-03-17',
 '2023-03-22',
 '2023-03-27',
 '2023-04-01',
 '2023-04-06',
 '2023-04-11',
 '2023-04-16',
 '2023-04-21',
 '2023-04-26',
 '2023-05-01',
 '2023-05-06',
 '2023-05-11',
 '2023-05-16',
 '2023-05-21',
 '2023-05-26',
 '2023-05-31',
 '2023-06-05',
 '2023-06-10',
 '2023-06-15',
 '2023-06-20',
 '2023-06-25',
 '2023-06-30',
 '2023-07-05',
 '2023-07-10',
 '2023-07-15',
 '2023-07-20',
 '2023-07-25',
 '2023-07-30',
 '2023-08-04',
 '2023-08-14',
 '2023-08-19',
 '2023-08-24',
 '2023-08-29',
 '2023-09-03',
 '2023-09-08',
 '2023-09-13',
 '2023-09-18',
 '2023-09-23',
 '2023-09-28',
 '2023-10-03',
 '2023-10-08',
 '2023-10-13',
 '2023-10-18',
 '2023-10-23',
 '2023-10-28',
 '2023-11-02',
 '2023-11-12',
 '2023-11-17',
 '2023-11-22',
 '2023-11-27',
 '2023-12-02',
 '2023-12-

In [16]:
import ipywidgets as widgets
from IPython.display import display

viz_params = {"min": 0, "max": 1.5, "gamma": 2.5}

# Create one map and display it once
coords = polygon.centroid().coordinates().getInfo()
lon, lat = coords
m = geemap.Map(center=[lat, lon], zoom=16)
display(m)

# Add initial layer
current_layer = m.addLayer(get_rgb_image(polygon, unique_dates[0]), viz_params, "RGB")

# Add initial date text
text_layer = m.add_text(
    unique_dates[0], position="topright", font_size=18, font_color="white", bold=True
)

slider = widgets.IntSlider(min=0, max=len(unique_dates)-1, value=0, description="Date")

def update_map(n):
    # Clear all layers (except basemap)
    for layer in list(m.layers)[1:]:  # skip base layer at index 0
        m.remove_layer(layer)

    # Add new RGB image
    rgb_image = get_rgb_image(polygon, unique_dates[n])
    m.addLayer(rgb_image, viz_params, "RGB")

    # Add date text
    m.add_text(unique_dates[n], position="topright", font_size=18, font_color="white", bold=True)

# Connect slider
slider.observe(lambda change: update_map(change["new"]), names="value")
display(slider)

Map(center=[0.9904466314006417, 34.986706752816545], controls=(WidgetControl(options=['position', 'transparent…

IntSlider(value=0, description='Date', max=70)